In [ ]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ast
import requests, json
import sys
sys.path.append( '/Users/bpb/Data/programming/OpenMSI/openmsi-programmatic-access/' )
import openmsi
import openmsi_spotset_tools
# plt.get_backend()
# plt.switch_backend('Qt4Agg')
client = requests.Session()

In [ ]:
# WHEN THIS BLOCK RUNS, YOU WILL HAVE TO ENTER YOUR PASSWORD INTO THE TERMINAL
client = openmsi.authenticateUser(client,'username')
# Test that we are authenticated by printing a list of files we have access to
myFiles = openmsi.getFilelist(client)
for k in myFiles:
    print k

In [ ]:
# myIons = [1241.25, 917.5, 817.6]
myIons = [2093.1, 2465.2, 2867.0, 3494.7, 3657.9, 5735.0]
myRange = 0.5 # integrate at +/- this amount
# filename = '/project/projectdirs/openmsi/omsi_data_private/bpb/20120913_nimzyme.h5'
filename = '/project/projectdirs/openmsi/omsi_data_private/raad0102/20150331MdR_MALDI-NIMS_1.h5'
dataIndex = '0'
expIndex = '0'
payload = {'file':filename,'format':'JSON','mtype':'file','expIndex':expIndex,'dataIndex':dataIndex}
url = 'https://openmsi.nersc.gov/openmsi/qmetadata'
r = client.get(url,params=payload)
metadata = json.loads(r.content)
originalSize = ast.literal_eval(metadata[u'children'][0][u'shape'])
print originalSize

mz = openmsi.getMZ(client,filename,expIndex,dataIndex)
imStack = np.zeros((originalSize[0],originalSize[1],len(myIons)))
for i,ion in enumerate(myIons):
    idx = np.where(abs(mz-ion)<myRange)
    payload = {'file':filename,
               'expIndex':expIndex,'dataIndex':dataIndex,'format':'JSON','mz':'%d:%d'%(min(idx[0]),max(idx[0]))}
    url = 'https://openmsi.nersc.gov/openmsi/qcube'
    r = client.get(url,params=payload)
    data = np.asarray(json.loads(r.content))
    imStack[:,:,i] = np.sum(data,2)

baseImage = np.sum(imStack,2)
plt.imshow(baseImage,cmap='CMRmap')
plt.show()

Set the circles where they need to be.  Then close the figure window. The last coordinates of the circles will be stored and you use them to study the interior points of your trapezoid.

In [ ]:
Nrows = 12
Ncols = 2
dragRadius = 4
pointMarkerSize = 12
xCenter,yCenter = openmsi_spotset_tools.roughPosition(baseImage,Nrows,Ncols,dragRadius,pointMarkerSize)

In [ ]:
markerRadius = 2
xCenter,yCenter = openmsi_spotset_tools.fineTunePosition(baseImage,xCenter,yCenter,markerRadius)

In [ ]:
# Save a dictionary into a pickle file.

import pickle

spotset_data = {'xCenter':xCenter,'yCenter':yCenter,
            'Nrows':Nrows,'Ncols':Ncols,
            'dragRadius':dragRadius,'pointMarkerSize':pointMarkerSize,'markerRadius':markerRadius,
            'baseImage':baseImage,'imStack':imStack,'myIons':myIons,'myRange':myRange,'mz':mz,'originalSize':originalSize,
            'filename':filename,'expIndex':expIndex,'dataIndex':dataIndex}

pickle.dump( spotset_data, open( "myWorkspace.pkl", "wb" ) )

In [ ]:
# Load the dictionary back from the pickle file.
import pickle

spotset_data = pickle.load( open( "myWorkspace.pkl", "rb" ) )
for key in spotset_data:
    globals()[key]=spotset_data[key]

In [ ]:
xEdges, yEdges = np.meshgrid(range(imStack.shape[1]), range(imStack.shape[0]), sparse=False, indexing='xy')
mask = np.zeros(baseImage.shape)
integrationRadius = 6 #distance to integrate
myPixels = []
for x,y in zip(xCenter,yCenter):
    idx = np.argwhere(((x - xEdges)**2 + (y - yEdges)**2)**0.5 < integrationRadius)
    myPixels.append(idx)
    for i in idx:
        mask[i[0],i[1]] = 1
plt.imshow(mask)
plt.show()
    

In [ ]:
minIntensity = 200

In [ ]:
fid = open('export.tab','wb')
fid.write('index\tfile\trow\tcolumn\trow-centroid\tcol-centroid\t')
for i in myIons:
    fid.write('%5.4f Sum\t' % i)
    fid.write('%5.4f Max\t' % i)
    fid.write('%5.4f Mean\t' % i)
    fid.write('%5.4f Median\t' % i)
    fid.write('%5.4f NumPixels\t' % i)
fid.write('\n')

for i,myPixel in enumerate(myPixels):
    fid.write('%d\t%s\t%s\t%s\t%d\t%d\t' % ( i, filename, 'coming_soon', 'coming_soon', np.mean(myPixel[:,0]), np.mean(myPixel[:,1]) ) )
    for i,ion in enumerate(myIons):
        values = []
        for j, coord in enumerate(myPixel):
            if imStack[coord[0],coord[1],i] > minIntensity:
                values.append(imStack[coord[0],coord[1],i])
        fid.write('%d\t%d\t%d\t%d\t%d\t' % (np.sum(values),np.max(values),np.mean(values),np.median(values),len(values)))
        
    fid.write('\n')
fid.close()